In [257]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import pandas as pd
import time
import openpyxl

input_search = '반바지'

url = "https://search.shopping.naver.com/search/all?query="+input_search
driver = webdriver.Chrome()
driver.get(url)
# driver.maximize_window()
time.sleep(5)

In [258]:
# 절대 위치로 스크롤
# driver.execute_script("window.scrollTo(0,500)")

In [259]:
# 상대 위치로 스크롤
# driver.execute_script("window.scrollBy(0,500)")

In [260]:
# 전체 스크롤 높이
# driver.execute_script("return document.body.scrollHeight")

In [261]:
# 페이지 끝까지 스크롤
# driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [262]:
# 무한 스크롤

# 마지막 스크롤 위치 받아오기
last_height = driver.execute_script("return document.body.scrollHeight")

# 무한스크롤
while True:    
    # 스크롤 끝까지 내리기
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    
    # 현재 스크롤 위치 받아오기
    current_height = driver.execute_script("return document.body.scrollHeight")  
    
    # 마지막스크롤 = 현재스크롤이면 반복 벗어나기  
    if last_height == current_height: break
    
    # 현재 스크롤을 마지막 스크롤로 업데이트 하기
    last_height = current_height
   
    time.sleep(5)
time.sleep(5)

### 데이터 수집

상품명, 상세페이지링크, 최저가, 별점, 리뷰수, 구매건수, 등록일

In [263]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

item_list = []
items = soup.select('.product_info_area__xxCTi')
print('상품갯수',len(items))
for item in items:

    product_name = item.select_one('.product_title__Mmw2K').text
    link = item.select_one('.product_link__TrAac.linkAnchor').attrs['href']
    price = item.select_one('.price_num__S2p_v').text.replace(',','').replace('원','')
    try:
        star_score = item.select_one('.product_grade__IzyU3').text.replace('별점','')
    except:
        star_score = ''
        
    try:
        review_cnt = item.select_one('.product_num__fafe5').text.replace('(','').replace(')','').replace(',','')
    except:
        review_cnt=''
    
    # 구매건수
    try:
        purchase = item.select_one('[data-nclick*=purchase] em').text.replace(',','')
    except:
        purchase = ''
        
    # 등록일
    elements = item.select('.product_etc__LGVaW')
    for element in elements:
        if '등록일' in element.text: reg_date=element.text.replace('등록일 ','')

    item_list.append([product_name, link, price, star_score, review_cnt, purchase, reg_date]) 

item_list

상품갯수 40


[['골든팜 인생닭 저염스팀 닭가슴살 오리지널 100g',
  'https://cr.shopping.naver.com/adcr.nhn?x=JzQPtcV%2BJxJsj3uloojskf%2F%2F%2Fw%3D%3Dsrj5lz9UbznZ9OS951%2FsZPIrjSGUMFEQs3zo1DDD6r46cONWhotBrwKALjmCx5HZ5D9mz4hsLv6ksGT4XM1MDHOTd5bo7kbUTdvuNP9h4W8NRHfuge9NqWnxke759%2Fd05OAEv8FlQtClxA%2BwCL5zXXUgAn8lD2%2BxrgblrBqYwPdWug%2B9sjA7rHdeMLkY%2BwIMU2G3a7XES977627%2FYLrD7DVAxMQ8erA10Vuapon9hanv6YmLBa42H8cL5v3nIZXXHlNfBkZd%2Fqnfn2S8yEGOGihtHciwKHxLAZj8KSHvyjLS0ZMS312IVQahwM3tj4oWNwc%2B%2FjkR2vl%2B%2B%2B%2F%2BVPNlrhUQ6cpKTqlmYGalnAZlcEQ24B4z81T0TMl3OC3Z20j9xOiSHso0SSRvAEEo5QU4QSvOCgpKxPkQRl1cLAa1tmh2qIjOdLKAWBvDXRPaixKMBIjbxAMh%2B10%2BjmxnveZkXVA5B2W8m8qDn1Y2%2ByWM35pT89xOAd04N8flksznIE2E%2FiMnwxsBow6MTGcWpjQS9gOp83EGd7MK7UBmIL2p2W%2BQ0ey71s8QTebS6g8kt8HLB2W3LOOrQsmTdnNdWl40N1%2FoLRdeTzJJo5DQ5ovPMjEUFmIpwQfAfAejWLvjQNUejoX8GuyGsHkDc62wn95eJgoVsN%2FiAcjmtIwlTUFZwg6A%3D&nvMid=27923095522&catId=50013800',
  '4490',
  '4.81',
  '4741',
  '',
  '2021.07.'],
 ['수지스 그릴드 닭가슴살 1.8kg',
  'https://cr.shopping.naver.com/adcr.nhn

### 파일로 저장

In [264]:
import time

# 파일명 생성 함수
def filename(input_search):
    save_datetime = time.localtime()
    year = save_datetime.tm_year
    mon = save_datetime.tm_mon
    mday = save_datetime.tm_mday
    hour = save_datetime.tm_hour
    min = save_datetime.tm_min
    sec = save_datetime.tm_sec
    save_datetime = f'{year}{mon:02d}{mday:02d}_{hour:02d}{min:02d}{sec:02d}'
    file_name = f'{input_search}_{save_datetime}'
    
    return file_name

# 파일 저장
import openpyxl

df = pd.DataFrame(item_list,
             columns = ['상품명','상세페이지url','최저가','별점','리뷰수','구매건수','등록일'])

df.to_excel(f'{filename(input_search)}.xlsx', index=False)
